In [1]:
import numpy as np
import pandas as pd
import wavelet_transform as wt
import signal_processing as sp
import scipy.sparse

/tmp/ipykernel_32802/2365126765.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [19]:
def precompute_wavelet(poly_division, time_interval, spatial = False):
    df = pd.read_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv")
    features = df.columns[2:]
    try:
        adj_matrix = np.load(f"data/adj_matrix/{poly_division}.npy")
    except:
        adj_matrix = scipy.sparse.load_npz(f"data/adj_matrix/{poly_division}.npz").toarray()

    signal_multi = np.stack(
        df.groupby("id_poly").apply(
            lambda x: x.sort_values("date").drop(columns=["date"]).values, include_groups=False
            ).values
        )
    
    n_timestamps = signal_multi.shape[1]
    if spatial:
        G_H = wt.get_spatial_adj(adj_matrix, n_timestamps)
    else:
        G_H = None

    df_res = []
    for k, feature in enumerate(features):
        df_start = df[["id_poly", "date"]].copy()
        signal = signal_multi[:, :, k]
        
        wav =  wt.WaveletTransform(
            adj_matrix,
            n_timestamps,
            G_H=G_H,
            graph_product="strong",
            n_filters=4,
            kernel="abspline",
            scaling_function=False,
            method="chebyshev",
            order_chebyshev=30,
        )
        coeffs = wav.transform(signal)
        coeffs = sp.get_scaled_coefficients(coeffs)


        # create column for coeffs in the df_res
        for i in range(coeffs.shape[-1]):
            df_start[f"mean_freq_{i}"] = coeffs[:, :, i].flatten()
        df_start["type"] = feature

        df_res.append(df_start)

    df_res = pd.concat(df_res)
    float_columns = df_res.columns[df_res.dtypes == np.float64] 
    df_res[float_columns] = df_res[float_columns].astype(np.float16)
    if spatial:
        df_res.to_csv(f"data/coeffs_spatial/{poly_division}_{time_interval}.csv", index=False)
    else:
        df_res.to_csv(f"data/coeffs/{poly_division}_{time_interval}.csv", index=False)

In [3]:
def precompute_both(poly_division, time_interval):
    precompute_wavelet(poly_division, time_interval)
    precompute_wavelet(poly_division, time_interval, True)

In [23]:
precompute_both("SpCenterCensus5k", "Period1")

In [24]:
precompute_both("SpCenterCensus5k", "Period2")

In [25]:
precompute_both("SpCenterCensus2k", "Period1")

In [26]:
precompute_both("SpCenterCensus2k", "Period2")

In [22]:
precompute_both("NYBlocks", "Period1")

In [28]:
precompute_both("BLACities", "Year")

In [30]:
precompute_both("BLACities", "Year2")